In [ ]:
import json
import torch
import torchaudio
from panns_inference import AudioTagging, labels

# === CONFIGURATIE ===
AUDIO_PATH = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Cake\Data\stabilizer pressure control.wav"
#AUDIO_PATH = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Cake\Data\Uitloopgoot.wav"
#AUDIO_PATH = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Cake\Data\machine cover.wav"
ONTOLOGY_PATH = "ontology.json"
CHECKPOINT_PATH = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\models\Audio\Cnn14_mAP=0.431.pth"
SEGMENT_SECONDS = 10
SAMPLE_RATE = 32000
TOOL_SCORE_THRESHOLD = 0.5
OUTPUT_JSON = "tool_detections.json"

# === TOOLS UIT ONTOLOGIE HALEN ===
with open(ONTOLOGY_PATH, 'r') as f:
    ontology = json.load(f)

id_to_node = {node['id']: node for node in ontology}
tools_id = next((node['id'] for node in ontology if node['name'].lower() == 'tools'), None)
if not tools_id:
    raise ValueError("Tools categorie niet gevonden in ontology.json")

def collect_tool_labels(node_id):
    node = id_to_node[node_id]
    collected = [node['name']]
    for child_id in node.get('child_ids', []):
        collected.extend(collect_tool_labels(child_id))
    return collected

tool_labels = set(label.lower() for label in collect_tool_labels(tools_id))

# === AUDIO VOORBEREIDEN ===
waveform, sr = torchaudio.load(AUDIO_PATH)
if sr != SAMPLE_RATE:
    waveform = torchaudio.functional.resample(waveform, sr, SAMPLE_RATE)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

total_samples = waveform.shape[1]
segment_samples = SEGMENT_SECONDS * SAMPLE_RATE
num_segments = total_samples // segment_samples

# === MODEL LADEN ===
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AudioTagging(checkpoint_path=CHECKPOINT_PATH, device=device)

# === SEGMENTEN ANALYSEREN ===
print(f"\nAnalyse van '{AUDIO_PATH}' in segmenten van {SEGMENT_SECONDS} seconden:")
results = []

for i in range(num_segments):
    start = i * segment_samples
    end = start + segment_samples
    segment = waveform[:, start:end]

    clipwise_output, _ = model.inference(segment)

    tool_hits = []
    for idx, label in enumerate(labels):
        if label.lower() in tool_labels:
            score = clipwise_output[0][idx].item()
            if score > TOOL_SCORE_THRESHOLD:
                tool_hits.append({
                    "label": label,
                    "score": round(score, 3)
                })

    if tool_hits:
        # Pak hoogste score
        best_hit = max(tool_hits, key=lambda x: x["score"])
        print(f"[{i*SEGMENT_SECONDS:>4}-{(i+1)*SEGMENT_SECONDS:>4} sec]")
        print(f"  - {best_hit['label']:25s}: {best_hit['score']:.3f}")

        results.append({
            "start_sec": i * SEGMENT_SECONDS,
            "end_sec": (i + 1) * SEGMENT_SECONDS,
            "tool_detected": best_hit
        })

# === JSON OPSLAAN ===
with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2)
    print(f"\nResultaten opgeslagen in '{OUTPUT_JSON}'")
